# XRD File reader and plotter

In [359]:
# Importamos librerias necesarias para nuestro programa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go  # Ensure plotly.graph_objects is imported
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import math

---

In [360]:
# Initialize Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    dcc.Input(id='y_position_input', type='number', value=0),  # Input box for y-position value
    dcc.Graph(id='line-plot')  # Graph area
])


---
## Se lee el archivo crudo

En esta parte del código se lee el archivo crudo proveniente del equipo de xrd y se trata para obtener una salida csv


In [361]:
file = open('../SrTiO3.uxd', mode='r')

content = file.read()

partes_importantes = content.split(';')

tabla_contenido = partes_importantes[7]

titles = tabla_contenido[1:15]

tabla_contenido = tabla_contenido.replace(titles, '2THETA, PSD\n')
tabla_contenido = tabla_contenido.replace('       ', ', ')
tabla_contenido = tabla_contenido.replace('      ', ', ')

file.close()

---
## Se convierte el archivo
En esta parte del código el contenido del archivo crudo se convierte en un archivo CSV para posteriormete abrirlo con pandas

In [362]:
output_file = open('data.csv', 'w')

output_file.write(tabla_contenido)

output_file.close()

In [363]:
df = pd.read_csv('data.csv')

referencia_horizontal = 157
y_position_value_global = 0


@app.callback(
    Output('line-plot', 'figure'),
    [Input('y_position_input', 'value')]
)
def update_plot(y_position_value):
    # Create an interactive plot

    global y_position_value_global  # Use the global variable
    y_position_value_global = y_position_value  # Store the value globally
    
    fig = px.line(df, x=' 2THETA', y=' PSD', labels={'Name': '2theta', 'Value': 'values'}, title='SrTiO3 difractograma')
    fig.update_traces(line=dict(color='blue')) 
   
    # Add a line parallel to x-axis at y_position_value
    fig.add_shape(
        type='line',
        x0=df[' 2THETA'].min(),
        y0=y_position_value,
        x1=df[' 2THETA'].max(),
        y1=y_position_value,
        line=dict(color='red', width=2, dash='solid'),
    )

    #-------------------------------------------------
    #Se calcula la altura máxima en de la reflexión más grande:
    altura_reflexion_max = max(df[' PSD'].values) - y_position_value_global

    given_PSD_value = altura_reflexion_max/2

    # Calculate absolute differences between given PSD value and all values in the 'PSD' column
    df['Absolute_Difference'] = abs(df[' PSD'] - given_PSD_value)

   # Find the row with the minimum absolute difference and get its index
    closest_index = df['Absolute_Difference'].idxmin()

    closest_2THETA_value = df.loc[closest_index, ' 2THETA']
    closest_PSD_value = df.loc[closest_index, ' PSD']

    print(f"The closest 2THETA value to PSD {given_PSD_value} is: {closest_2THETA_value} (PSD: {closest_PSD_value})")
    print(f"Index of the closest value: {closest_index}")

    # Find the next upper closest 2THETA value
    next_upper_values = df[df[' PSD'] > given_PSD_value]
    if not next_upper_values.empty:
        next_upper_index = next_upper_values[' PSD'].idxmin()
        next_upper_2THETA = df.loc[next_upper_index, ' 2THETA']
        next_upper_PSD = df.loc[next_upper_index, ' PSD']
    
        print(f"The next upper closest 2THETA value to PSD {given_PSD_value} is: {next_upper_2THETA} (PSD: {next_upper_PSD})")
        print(f"Index of the next upper closest value: {next_upper_index}")
    else:
        print("No next upper value found.")
    #-------------------------------------------------
    
    # Add two points (indices 0 and 1) as annotations
    fig.add_trace(go.Scatter(x=[df[' 2THETA'][closest_index]], y=[df[' PSD'][closest_index]], mode='markers+text', text='Point A', textposition='bottom center', marker=dict(color='blue')))
    fig.add_trace(go.Scatter(x=[(df[' 2THETA'][next_upper_index] + df[' 2THETA'][next_upper_index + 1])/2], y=[df[' PSD'][closest_index]], mode='markers+text', text='Point B', textposition='bottom center', marker=dict(color='red')))

    # Calculate distance between two points (using Euclidean distance)
    distance = math.sqrt((df[' 2THETA'][461] - df[' 2THETA'][482])**2 + (df[' PSD'][461] - df[' PSD'][482])**2)

    # Add annotation with the calculated distance
    fig.add_annotation(
        x=2,
        y=4,
        text=f'Distance: {distance:.2f}',  # Display distance with two decimal points
        showarrow=True,
        arrowhead=1,
    )

    return fig


In [364]:
#update_plot(200)

In [365]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

The closest 2THETA value to PSD 18187.5 is: 31.8604 (PSD: 17860)
Index of the closest value: 466
The next upper closest 2THETA value to PSD 18187.5 is: 32.1419 (PSD: 21608)
Index of the next upper closest value: 472


In [264]:
lista_de_angulos_2theta = df[' 2THETA'].values
lista_de_angulos_theta = []
for angulo in lista_de_angulos_2theta:
    lista_de_angulos_theta.append(angulo/2)

In [186]:
sen_angulos = []
cos_angulos = []
for angulo in lista_de_angulos_theta:
    sen_angulos.append(np.sin(angulo))
    cos_angulos.append(np.cos(angulo))